In [1]:
string = 'Here is a string of text'

In [2]:
from big_phoney import BigPhoney

Using TensorFlow backend.


In [3]:
phoney = BigPhoney()

In [4]:
phoney.count_syllables(string)

6

In [6]:
from big_phoney import BigPhoney
import lyricsgenius as genius
import random
import nlpaug.augmenter.word as naw
#from nlpaug.util import Action
import re

In [7]:
def get_target_lyrics(artist, title):
    
    api_key = 'YOUR_API_KEY_HERE'
    api = genius.Genius(api_key)
    api.remove_section_headers = True # Removes section headers like "Verse" and "Intro"
    
    #get song lyrics
    song = api.search_song(artist, title)
    
    #clean up the target lyrics
    target = song.lyrics
    target = target.replace("\u2005"," ")
    target = target.replace("\\"," ")
    target = target.replace("\\n"," ")
    target = target.replace("("," ")
    target = target.replace(")"," ")
    target = target.replace("\n\n","\n")
    target = target.replace("\n\n\n","\n")
    target = target.replace("2x","")
    #create list of lines
    target_lyrics = target.split("\n")
    #print("Target lyrics:")
    #print(target_lyrics)
    return target_lyrics

In [8]:
def lyrics_to_list(file_name):
    with open(file_name, 'r') as file:
        data = file.read().split("====================")
        random_song = random.choice(data)
        gen_lyrics = random_song.split("\n")
        gen_lyrics = gen_lyrics[1:-1]
#    print(gen_lyrics)
    return gen_lyrics

In [9]:
def fit_lyrics(gen_lyrics, target_lyrics):
    print(gen_lyrics)
    aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
    print('aug')
    phoney = BigPhoney()
    print('initialized phoney')
    def count_syls(text):
    
        schema = []
    
        for line in text:
            syls = phoney.count_syllables(line)
            schema.append(syls)
            #print(syls,line)
        
        return schema
    
    def decontracted(phrase):
        # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase
    
    gen_schema = count_syls(gen_lyrics)
    target_schema = count_syls(target_lyrics)
#    print(target_schema)
    #make generated lyrics same length as target lyrics
    target_len = len(target_schema)
    del gen_schema[target_len:]
    del gen_lyrics[target_len:]
    new_lyrics = []
#    loop = 1
    for num, line in enumerate(gen_lyrics):
        
        if (gen_schema[num] == target_schema[num]): 
            new_lyrics.append(line)
        elif (gen_schema[num] != target_schema[num]):
            syls = gen_schema[num]
            track = 0
            while syls != target_schema[num]:
                #while syls don't match the target
                #make syls the next syls 
                #if we're at the end then stop 
                if track == len(gen_schema)-1:
                    end = True
                    break
                else:
                    track = track + 1
                    syls = gen_schema[track]
                    line = gen_lyrics[track]
                    end = False
            if end == False:
                new_lyrics.append(line)
            #If we can't find a line that's the right size
            if end:
                line = decontracted(line)
#                print("No matching line")
                #print(line)
                while syls < target_schema[num]:
#                    print("not enough syls")
                    original_line = line
                    line = aug.augment(line)
#                    print(line)
                    syls = phoney.count_syllables(line)
                #In case we overshoot (add too many syllables)
                    if syls > target_schema[num]:
                        #print("Oops we overshot")
                        line = original_line
                        syls = phoney.count_syllables(line)
                new_line = line
#                print("the same line:")
#                print(line)
                syls = gen_schema[num]
                words = line.split(" ")
                while syls > target_schema[num]:
#                    print("too many syls")
                    del words[-1]
                    new_line = ' '.join(words)
                    syls = phoney.count_syllables(new_line)
                    #In case too many syllables are deleted
                    if syls < target_schema[num]:
                        #print("Oops, deleted too many")
                        new_line = aug.augment(new_line)
                        syls = phoney.count_syllables(new_line)
                new_lyrics.append(new_line)
    return new_lyrics

In [10]:
target_lyrics = get_target_lyrics('Idina Menzel','Let it go')
gen_lyrics = target_lyrics
new_lyrics = fit_lyrics(gen_lyrics,target_lyrics)
print(new_lyrics)

Searching for "Idina Menzel" by Let it go...
Done.
['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, Heaven knows I tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on!', 'The cold never bothered me anyway', "It's funny how some distance", 'Makes everything seem small', 'And the fears that once controlled me', "Can't get to me at all", "It's time to see what I can do", 'To test the limits and break through', 'No right, no wrong, no rules for me', "I'm free", 'Let it go, let it go', "I'm one with the wind and sky", 'Let it go, let it go', "You'll n